In [18]:
import pandas as pd
import json

In [19]:
iteration_path = "iteration_3/"
deliveries_path = "../data/" + iteration_path + "raw/deliveries.jsonl"
products_path = "../data/" + iteration_path + "raw/products.jsonl"
sessions_path = "../data/" + iteration_path + "raw/sessions.jsonl"
users_path = "../data/" + iteration_path + "raw/users.jsonl"

deliveries_data = pd.read_json(deliveries_path, lines=True)
products_data = pd.read_json(products_path, lines=True)
sessions_data = pd.read_json(sessions_path, lines=True)
users_data = pd.read_json(users_path, lines=True)

In [20]:
def merge_to_json(users: pd.DataFrame, sessions: pd.DataFrame, products: pd.DataFrame, deliveries: pd.DataFrame):
    users = json.loads(users.to_json(orient='records'))
    sessions = json.loads(sessions.to_json(orient='records', date_unit='ms'))
    products = json.loads(products.to_json(orient='records'))
    deliveries = json.loads(deliveries.to_json(orient='records'))
    json_object = {"users": users, "sessions": sessions, "products": products, "deliveries": deliveries}
    save_to_file("ab_test/ab_test_data.json", json_object)

def save_to_file(output, json_object):
    with open(output, "w+", encoding="utf-8") as file:
        file.write(json.dumps(json_object, indent=4, sort_keys=True))

In [21]:
train_sessions_data = sessions_data[sessions_data['timestamp'].dt.month <= 11]
merge_to_json(users_data, train_sessions_data, products_data, deliveries_data)

In [22]:
from features.build_features import aggregate_users_data

test_sessions_data = sessions_data
extracted_users_data = aggregate_users_data(test_sessions_data, users_data, products_data)
extracted_users_data["expenses"] = extracted_users_data["expenses"].map(lambda x: round(x, 2))
extracted_users_dict = extracted_users_data[["user_id", "expenses"]].to_dict('split')
json_object = {int(user_id): expenses for user_id, expenses in extracted_users_dict["data"]}
save_to_file("ab_test/ab_test_good_data.json", json_object)